In [ ]:
%%file AIFlyLocal.py
# -*- coding:utf-8 -*-
from model_loader import ModelLoader
import json
import sys
import logging
from AIFlysdk.AIFlyApi import AIFlyApi
from model_loader import ModelLoader
import yaml

class PredictionInput:
    """
    Contains input details and model_id and data
    model_id:'tensormodel'
    model_version:'0.1.0
    data:
        - input:数据输入
    """  
    def __init__(self,model_id,model_version,data):
        self.model_id=model_id
        self.model_version=model_version
        self.data=data

class ModelInput:
    """
    Contains input details and model_id and data
    funcslist:['predpy.pred']
        - predpy:文件名
        - pred:函数名
    """  
    def __init__(self,funcslist):
        self.funcslist=funcslist
class PredictServer(object):
    def __init__(self,config):
        self.logger = logging.getLogger(__name__)
        self.conf=config#'config/AIFlyapi_config.yaml'
        self.models_loaded={}
        with open(self.conf, 'rt') as f:
            self.client_config = yaml.load(f.read())
        self.models_loaded = {}#model_loader.get_models_from_list(models_to_load)

    def load_models(self, model_list):
        """Handles deployed model for init"""
        self.logger.debug('Update models received with payload: %s', str(model_list))
        model_loader = ModelLoader(self.client_config)
        models_to_load=model_list
        self.models_loaded = model_loader.get_models_from_list(models_to_load)
        #model_list=[["HelloWorldExample", "1.0.0"],["HelloAI2", "1.0.0"],["TensorflowMnistExample","1.0.0"],['fib_model',"1.0.2"]]
        
        return json.dumps({"Init Model Service": "Success"})
    def predict(self, payload):
        if ('data' in payload.keys()) and ('modelId' in payload.keys()):
            PredictionInput.data = payload['data']
            PredictionInput.model_id = payload['modelId']
            PredictionInput.model_version=payload['model_version']
            key = (PredictionInput.model_id, PredictionInput.model_version)
            curr_model = self.models_loaded[key]
             
        else:           
            print("Bad Request", "model_version and(or) modelId missing in the payload")
        try:
            po = json.dumps({'result':curr_model.predict(PredictionInput.data)})
            return po
        except:
            return json.dumps({'status': 'Failure', 'message' : 'Error occurred'})

In [4]:
%%file AIFlylocal/AIFlylocal.py
# -*- coding:utf-8 -*-
from model_loader import ModelLoader
import json
import sys
import logging
from AIFlysdk.AIFlyApi import AIFlyApi
from model_loader import ModelLoader
import yaml

class PredictionInput:
    """
    Contains input details and model_id and data
    model_id:'tensormodel'
    model_version:'0.1.0
    data:
        - input:数据输入
    """  
    def __init__(self,model_id,model_version,data):
        self.model_id=model_id
        self.model_version=model_version
        self.data=data

class ModelInput:
    """
    Contains input details and model_id and data
    funcslist:['predpy.pred']
        - predpy:文件名
        - pred:函数名
    """  
    def __init__(self,funcslist):
        self.funcslist=funcslist
class PredictServer(object):
    def __init__(self,config):
        self.logger = logging.getLogger(__name__)
        self.conf=config#'config/AIFlyapi_config.yaml'
        self.models_loaded={}
        with open(self.conf, 'rt') as f:
            self.client_config = yaml.load(f.read())
        self.models_loaded = {}#model_loader.get_models_from_list(models_to_load)

    def load_models(self, model_list):
        """Handles deployed model for init"""
        self.logger.debug('Update models received with payload: %s', str(model_list))
        model_loader = ModelLoader(self.client_config)
        models_to_load=model_list
        self.models_loaded = model_loader.get_models_from_list(models_to_load)
        #model_list=[["HelloWorldExample", "1.0.0"],["HelloAI2", "1.0.0"],["TensorflowMnistExample","1.0.0"],['fib_model',"1.0.2"]]
        
        return json.dumps({"Init Model Service": "Success"})
    def predict(self, *args,**kwargs):
        #PredictionInput.data = *args
        if ('model_version' in kwargs.keys()) and ('modelId' in kwargs.keys()):   
            PredictionInput.model_id = kwargs['modelId']
            PredictionInput.model_version=kwargs['model_version']
            key = (PredictionInput.model_id, PredictionInput.model_version)
            curr_model = self.models_loaded[key]
             
        else:           
            print("Bad Request", "model_version and(or) modelId missing in the payload")
        try:
            po = json.dumps({'result':curr_model.predict(*args)})
            return po
        except:
            return json.dumps({'status': 'Failure', 'message' : 'Error occurred'})

Overwriting AIFlylocal/AIFlylocal.py


In [7]:
model_local_server=PredictServer('config/AIFlyapi_config.yaml')
load_model=model_local_server.load_models([["fib_model", "1.0.2"]])

In [19]:
input = {'modelId':'fib_model','model_version':'1.0.2', 'data':{'n':2}}

model_local_server.predict(input)

'{"status": "Failure", "message": "Error occurred"}'

In [1]:
from AIFlylocal.AIFlylocal import PredictServer
model_local_server=PredictServer('config/AIFlyapi_config.yaml')
load_model=model_local_server.load_models([["fib_model", "1.0.2"]])

param = {'modelId':'fib_model','model_version':'1.0.2'}


model_local_server.predict(21,modelId='fib_model',model_version='1.0.2')

'{"result": 17711}'

In [42]:
def request(**kwargs):
    data, files = decouple_files(kwargs)
    print data, files 
def decouple_files(kwargs):
    data = {arg: value for arg, value in kwargs.items() if not is_file(value)}
    files = {arg: value for arg, value in kwargs.items() if is_file(value)}
    return data, files

def is_file(value):
    return hasattr(value, 'read') or hasattr(value, 'readlines')
request(c=1)

{'c': 1} {}


In [33]:

def test(a,*args,**kwargs):
    print a
    #print b
    #print c
    print args
    print kwargs,kwargs.keys()
test(1,[3,13],'d',d='4',e=5)


1
([3, 13], 'd')
{'e': 5, 'd': '4'} ['e', 'd']


In [44]:
def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)
def test(*args):
    print args
    print fib(*args)
test(3,2)    

(3, 2)


TypeError: fib() takes exactly 1 argument (2 given)

In [45]:
def fib(x,y):
    return x+y
def test(*args):
    print args
    print fib(*args)
test(3,2) 

(3, 2)
5


In [14]:
import AIFlyLocal

In [ ]:
import logging
import time

logger = logging.getLogger(__name__)
class Client:
    def __init__(self, server_url, auth_token=None):
        # strip trailing / to avoid double / chars in the URL
        self.server_url = server_url.rstrip("/")
        self.auth_token = auth_token
        self._metadata = None

    def __getattr__(self, func_name):
        return RemoteFunction(self, func_name)

    def call_func(self, func_name, **kwargs):
        path = self._get_path(func_name)
        return self.request(path, **kwargs)
    def request(self, _path, **kwargs):
        url = self.server_url + _path
        t0 = time.time()
        try:
            headers = self.prepare_headers()
            data, files = self.decouple_files(kwargs)
            if files:
                response = requests.post(url, data=data, files=files, headers=headers, stream=True)
            else:
                response = requests.post(url, json=data, headers=headers, stream=True)
        except ConnectionError:
            raise FireflyError('Unable to connect to the server, please try again later.')
        finally:
            t1 = time.time()
            logger.info("%0.3f: POST %s", t1-t0, url)
            return self.handle_response(response)
    def decouple_files(self, kwargs):
        data = {arg: value for arg, value in kwargs.items() if not self.is_file(value)}
        files = {arg: value for arg, value in kwargs.items() if self.is_file(value)}
        return data, files

    def is_file(self, value):
        return hasattr(value, 'read') or hasattr(value, 'readlines')

def RemoteFunction(client, func_name):
    def wrapped(*args, **kwargs):
        if args:
            raise AIflyError('Firefly functions only accept named arguments')
        return client.call_func(func_name, **kwargs)
    wrapped.__name__ = func_name
    wrapped.__qualname__ = func_name
    wrapped.__doc__ = client.get_doc(func_name)
    return wrapped

class AIflyError(Exception):
pass